In [ ]:
# COOKIE
> Javascript Cookies

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: 

## How to make the game

The basic html consists of 2 parts. The canvas for the game and the scoring part. The canvas part is put in one div, whereas the scoring is put in another. 

In [ ]:
<body>
  <div class="container">
    <div class="canvas-container">
      <canvas id="game" width="375" height="667"></canvas>
    </div>
    <div class="info-container">
      <div id="score">Score: </div>
      <div id="highestScore">Highest Score: </div>
    </div>
    <div class="table">
      <table id="tablearr">
        <!-- Table headers or initial content -->
        <tr>
          <th>Session</th>
          <th>Score</th>
        </tr>
      </table>
    </div>
  </div>
</body>

## The game part

In this game, we need a variable to store the score. The game also requires a canvas to make the doodle move. Then, we need a object for the "doodle", the moving character. We also need an object for the platforms. 

In [ ]:
class Platform { //platforms
    constructor(x, y) {
        this.x = x;
        this.y = y;
    }
    get X() {
        return this.x;
    }
    get Y() {
        return this.y;
    }
    set X(x)
    {
      this.x=x;
    }
    set Y(y)
    {
      this.y=y;
    }
} 
class Doodle { //the game character
constructor(x, y, dx, dy) {
  this.x = x;
  this.y = y;
  this.dx = dx;
  this.dy = dy;
  this.width = 40;
  this.height = 60;
}
get W() {
  return this.width;
}
get H() {
  return this.height;
}
get X() {
  return this.x;
}
get Y() {
  return this.y;
}
set X(x) {
  this.x = x;
}
set Y(y) {
  this.y = y;
}
get Dx() {
  return this.dx;
}
get Dy() {
  return this.dy;
}
set Dx(dx) {
  this.dx = dx;
}
set Dy(dy) {
  this.dy = dy;
}
}

function random(min, max) {
return Math.random() * (max - min) + min; //random platforms
}
//set values for the platforms
const canvas = document.getElementById('game');
const context = canvas.getContext('2d');
// width and height of each platform and where platforms start
const platformWidth = 65;
const platformHeight = 20;
const platformStart = canvas.height - 50; //platformStart - 617
// player physics
const gravity = 0.33;
const drag = 0.3;
const bounceVelocity = -12.5;
//window varible
var windowId = undefined;
//score counter and variable
var score = 0;
var highscore = getHighScore(); //highscore
//
// minimum and maximum vertical space between each platform
let minPlatformSpace = 15;
let maxPlatformSpace = 20;
//* starting adding platforms to the canvas 
let y = platformStart;
let doodlePlatforms = [new Platform(canvas.width / 2 - platformWidth / 2, platformStart)];
//doodle image
var dimg = new Image();
dimg.src = "https://raw.githubusercontent.com/JasonMize/coding-league-assets/master/doodle-jump-doodler.png";
//platforms image
var pimg = new Image();
pimg.src = "platform3.png";
//background image
var bimg = new Image();
bimg.src = "dbackground(1).png";
//create platforms until y reaches
while (y > 0) {
// the next platform can be placed above the previous one with a space
// somewhere between the min and max space
y -= platformHeight + random(minPlatformSpace, maxPlatformSpace); 
//suppose it is y = 595 when called
// a platform can be placed anywhere 25px from the left edge of the canvas
// and 25px from the right edge of the canvas (taking into account platform
// width).
// however the first few platforms cannot be placed in the center so
// that the player will bounce up and down without going up the screen
// until they are ready to move
let x;
do {
  x = random(25, canvas.width - 25 - platformWidth); //x = 259 suppose
} while (
  y > canvas.height / 2 &&
  x > canvas.width / 2 - platformWidth * 1.5 &&
  x < canvas.width / 2 + platformWidth / 2
);
doodlePlatforms.push(new Platform(x, y))
}

let doodle = new Doodle(canvas.width / 2 - 20, platformStart - 60, 0, 0);
// keep track of player direction and actions
let playerDir = 0;
let keydown = false;
let prevDoodleY = doodle.Y;
//game loop
function loop() {
//updateScore();
//check if the doodle falls off
if(doodle.Y > canvas.height) {
  alert("Doodle fell off, game over!!!!!");
  if (score > highscore) {
    highscore = score;
    document.getElementById('highestScore').innerHTML = "Highscore: "+ highscore;
    document.cookie = 'highscore' + '=' + score + '; expires=' +    daysToExpire + ';SameSite=None';
    console.log(highscore);
  }
  document.getElementById('score').innerHTML = "Score: "+score;
  cancelAnimationFrame(windowId); 
  createNewCookie();
  location.reload();       
  return;
}
windowId = undefined;
requestAnimationFrame(loop);
//context.clearRect(0,0,canvas.width,canvas.height);
//drawing background from image
context.drawImage(bimg, 0, 0, canvas.width,canvas.height);
// apply gravity to doodle
doodle.Dy += gravity;
// if doodle reaches the middle of the screen, move the platforms down
// instead of doodle up to make it look like doodle is going up
if (doodle.Y < canvas.height / 2 && doodle.Dy < 0) {
  doodlePlatforms.forEach(function(platform) {
    platform.Y += -doodle.Dy;
  });
  // add more platforms to the top of the screen as doodle moves up
  while (doodlePlatforms[doodlePlatforms.length - 1].Y > 0) {
    doodlePlatforms.push(new Platform(random(25, canvas.width - 25 - platformWidth),doodlePlatforms[doodlePlatforms.length - 1].Y - (platformHeight + random(minPlatformSpace, maxPlatformSpace))));
    score++;
    document.getElementById('score').innerHTML = "Score: "+score;
    // add a bit to the min/max platform space as the player goes up
    minPlatformSpace += 0.5;
    maxPlatformSpace += 0.5;
    // cap max space
    maxPlatformSpace = Math.min(maxPlatformSpace, canvas.height / 2);
  }
}
else {
  doodle.Y += doodle.Dy;
}
// only apply drag to horizontal movement if key is not pressed
if (!keydown) {
  if (playerDir < 0) {
    doodle.Dx += drag;
    // don't let dx go above 0
    if (doodle.Dx > 0) {
      doodle.Dx = 0;
      playerDir = 0;
    }
  }
  else if (playerDir > 0) {
    doodle.Dx -= drag;
    if (doodle.Dx < 0) {
      doodle.Dx = 0;
      playerDir = 0;
    }
  }
}
doodle.X += doodle.Dx;
// make doodle wrap the screen
if (doodle.X + doodle.W < 0) {
  doodle.X = canvas.width;
}
else if (doodle.X > canvas.width) {
  doodle.X = -doodle.W;
}
// draw platforms
context.fillStyle = 'green';
doodlePlatforms.forEach(function(platform) {
  //context.fillRect(platform.X, platform.Y, platformWidth, platformHeight);
  context.drawImage(pimg, platform.X, platform.Y, platformWidth, platformHeight);
  // make doodle jump if it collides with a platform from above
  if (
    // doodle is falling
    doodle.Dy > 0 &&
    // doodle was previous above the platform
    prevDoodleY + doodle.H <= platform.Y &&
    // doodle collides with platform
    // (Axis Aligned Bounding Box [AABB] collision check)
    doodle.X < platform.X + platformWidth &&
    doodle.X + doodle.W > platform.X &&
    doodle.Y < platform.Y + platformHeight &&
    doodle.Y + doodle.H > platform.Y
  ) {
    // reset doodle position so it's on top of the platform
    doodle.Y = platform.Y - doodle.H;
    doodle.Dy = bounceVelocity;
  }
});
// draw doodle
context.fillStyle = 'yellow';
//context.fillRect(doodle.X, doodle.Y, doodle.W, doodle.H);
context.drawImage(dimg, doodle.X, doodle.Y, doodle.W, doodle.H);
prevDoodleY = doodle.Y;
// remove any platforms that have gone offscreen
doodlePlatforms = doodlePlatforms.filter(function(platform) {
  return platform.Y < canvas.height;
})
}
// listen to keyboard events to move doodle
document.addEventListener('keydown', function(e) {
// left arrow key
if (e.which === 37) {
  keydown = true;
  playerDir = -1;
  doodle.Dx = -3;
}
// right arrow key
else if (e.which === 39) {
  keydown = true;
  playerDir = 1;
  doodle.Dx = 3;
}
});
document.addEventListener('keyup', function(e) {
keydown = false;
});
// start the game
windowId = requestAnimationFrame(loop);
//updateScore();

## What are Cookies

Cookies are information that is stored locally in the browser. Since it is stored locally, the data will still be there even if you close the browser or reload the session. 

However, there are some cons of cookies. Namely, there isn't a lot of storage space allocated for cookies. Cookies are also a privacy concern because they store potentially private information. 

## Main uses of Cookies in the case of a game

In this game, we used cookies to store the scores of each play through. In order to do this, we need to know how to create cookies, as well as read cookies. 

In [ ]:
document.cookie = "cookiename=value1";

### Creating Cookies

The "cookiename" is the name of the cookie, and the "value1" is the value of the cookie. However, by default, the session will delete the cookie when you close the browser. In order to stop this, a expiration date is needed. 

In [ ]:
document.cookie = "cookiename=value1; expires=Thu, 18 Dec 2038 12:00:00 UTC";

### Reading Cookies


A variable can be set to the name and the value of the cookie. In this example, the name, value, and the expiration date are all variables. 

When a variable is set to document.cookie, it will return all cookies in the way of "cookie1=value1; cookie2=value2;" and so on.

In [ ]:
let x = document.cookie = cookieName + '=' + cookieValue + '; expires=' + daysToExpire + ';SameSite=None';

var cookieString = document.cookie; //returns all cookies


### Deleting cookies

To delete a cookie, just set the expiration date to a time before the current time. 

function deleteCookie(cookieName) {
      document.cookie = cookieName + '=; expires=Thu, 01 Jan 1970 00:00:00 UTC; path=/;';
  }

### Creating a table

To create a table containing all the cookie values, you would want to create a for loop that loops through all the values stored. After that, you create a column for the name, and another column for the value. 


In [ ]:
var tableBody = document.querySelector('#tablearr'); //pushes onto the table


function tablegen() {
    for (var i = 0; i < cookies.length; i++) {
      var cookie = cookies[i].trim().split('='); //trim and split it up before and after the equal sign
      var name = cookie[0]; //first half
      var value = cookie[1]; //second half
      
      var row = document.createElement('tr');
      var nameCell = document.createElement('td');
      nameCell.textContent = name;
      row.appendChild(nameCell); //adds the item
  
      var valueCell = document.createElement('td');
      valueCell.textContent = value; //value of the cookie
      row.appendChild(valueCell);
      tableBody.appendChild(row);
      
      var deleteButtonCell = document.createElement('td');
      var deleteButton = document.createElement('button'); //the butotn
      deleteButton.textContent = 'rmove'; //text in button
      deleteButton.addEventListener('click', function()
      {
        deleteCookie(name); //functino so that it doesn't run automatically
      });
      deleteButtonCell.appendChild(deleteButton);
      row.appendChild(deleteButtonCell); //these 2 add the button
    }
  }

<table id="tablearr">
  <!-- Table headers or initial content -->
  <tr>
    <th>Session</th>
    <th>Score</th> 
    <!-- in this example, there are only 2 columns, but if you wanted to have a third for the delete, just create another <th> tag.  -->
  </tr>
</table>


### High Score

In the game, we had a high score function. We just created a cookie highscore, that everytime that the game reset, it would check if the current score was greater than the highscore, in which we replaced the highscore with that score. 

In order to return the high score, we had to have a function that would look for that highscore.

In [ ]:
function getHighScore() {
    var cookieString = document.cookie; //returns all cookies
    var cookieArray = cookieString.split(";"); //splits by ;, so it goes by each cookie.
  
    // Iterate over each cookie
    for (var i = 0; i < cookieArray.length; i++) {
      var cookie = cookieArray[i].trim();
  
      // Check if the cookie starts with the provided name
      if (cookie.startsWith("highscore=")) {
        // Extract and return the cookie value
        var score = parseInt(cookie.substring("highscore=".length));    
        return score;
        }
    }
  
    // Return null if the cookie is not found
    console.log("notfound");
    return null;
  }